In [125]:
import pandas as pd
import config 
import numpy as np

arg = config.get()

In [126]:
train = pd.read_csv("train.csv", encoding="cp949")
test = pd.read_csv("test.csv", encoding="cp949")

In [127]:
import random
from collections import Counter

def get_label_dist(df):
    dist_df = dict(Counter(df["label"]))

    total = len(df)

    for key, value in dist_df.items():
        dist_df[key] = np.round(value/total, 2)
        
    return dist_df

def sampling_by_dist(train, test, size):

    val_vocid = []
    vocids = train["vocid"]
    label_list = list(set(train["label"]))
    dist = get_label_dist(test)

    for label in label_list:

        ratio = dist[label]
        target_cnt = int(ratio*size)

        pop_list = list(train[train["label"]==label]["vocid"])

        val_vocid += random.sample(pop_list, target_cnt)
        
    val_df = train[train["vocid"].isin(val_vocid)]
    train_df = train[~train["vocid"].isin(val_vocid)]

    return train_df, val_df

In [128]:
train_df, val_df = sampling_by_dist(train, test, 300)

In [166]:
import copy

def RS(df, aug_size, loop_cnt):
    
    if aug_size > len(df):
        tmp = copy.deepcopy(df)
        
    else :
        aug_vocid = random.sample(list(df["vocid"]), aug_size)
        tmp = df[df['vocid'].isin(aug_vocid)]
        
        
    ####################
    #RS구현
    ####################
    df = pd.concat([df, tmp], axis=0)

    return df

def get_aug_data(df, target_size):
    
    aug_data = pd.DataFrame()
    print(target_size)
    print("=")
    print(len(df))
    
    loop_cnt = 0
    
    while True:
        loop_cnt += 1
        aug_size = target_size - len(df)
        print(str(loop_cnt) + "th aug_size : ", str(aug_size))

        if aug_size > 0:
            
            df = RS(df, aug_size, loop_cnt)
        
        elif aug_size == 0 :
            return df
            
        else: 
            aug_vocid = random.sample(list(df["vocid"]), target_size)
            return df[df['vocid'].isin(aug_vocid)]
            

        

In [167]:

def adjust_dist(train, test):

    label_list = list(set(train["label"]))
    dist = get_label_dist(test)
    train_size = 10000

    aug_train_df = pd.DataFrame()

    for label in label_list:

        print("label :", label)

        ratio = dist[label]
        target_size = int(ratio*train_size)

        df = train[train["label"]==label]
        aug_train_df = pd.concat([aug_train_df, get_aug_data(df, target_size)], axis=0)
        
    return aug_train_df



In [168]:
adjust_dist(train_df, test)

label : A
3100
=
1063
1th aug_size :  2037
2th aug_size :  974
3th aug_size :  -520
label : C
4900
=
282
1th aug_size :  4618
2th aug_size :  4336
3th aug_size :  3772
4th aug_size :  2644
5th aug_size :  388
6th aug_size :  -3116
label : B
2000
=
619
1th aug_size :  1381
2th aug_size :  762
3th aug_size :  -274


,vocid,label,colA,colB
1,2000002,A,0.101638,0.672577
3,2000004,A,0.365941,0.767438
4,2000005,A,0.248131,0.887697
5,2000006,A,0.136992,0.355926
8,2000009,A,0.406436,0.956373
...,...,...,...,...
2245,2002246,B,0.914023,0.843585
2248,2002249,B,0.787488,0.318605
2252,2002253,B,0.863666,0.495997
2256,2002257,B,0.722631,0.010069
